In [55]:
import pandas as pd
import numpy as np



In [56]:
import pandas as pd

# 文件名，通常是MapReduce任务输出结果文件之类的，这里假设是处理后的包含交易相关数据的文件
file_name = "part-r-00000"
# 定义列名列表，对应最终要构建的数据表中各列的名称，涵盖了不同规模买卖单的成交量、成交额以及主力资金流向等指标和时间范围信息
columns_list = [
    "主力净流入",
    "主力流入",
    "主力流出",
    "超大买单成交量",
    "超大买单成交额",
    "超大卖单成交量",
    "超大卖单成交额",
    "大买单成交量",
    "大买单成交额",
    "大卖单成交量",
    "大卖单成交额",
    "中买单成交量",
    "中买单成交额",
    "中卖单成交量",
    "中卖单成交额",
    "小买单成交量",
    "小买单成交额",
    "小卖单成交量",
    "小卖单成交额",
    "时间范围",
]

# 用于存储所有出现过的时间信息，方便后续判断时间是否已存在等操作
time_list = []

# 创建一个空的DataFrame对象，使用定义好的列名列表来初始化列，后续会逐步填充数据
price_table = pd.DataFrame(columns=columns_list)

# 打开指定的文件，使用上下文管理器确保文件使用完毕后正确关闭
with open(file_name) as f:
    # 循环读取文件中的每一行，直到文件末尾（读取到空行）
    while True:
        line = f.readline()
        if not line:
            break
        # 按制表符（\t）分割每行数据，通常是将键和值部分分开（假设数据格式是这样组织的）
        separate_by_space = line.split("\t")
        # 对分割后的第二个元素（通常是值部分，假设值内是以逗号分隔不同字段的格式）再按逗号进行分割，获取具体的数值字段
        separate_by_colon = separate_by_space[1].split(",")
        # 对分割后的第一个元素（通常是键部分，假设键内是以"_"分隔不同字段的格式）再按"_"进行分割，获取如订单类型、订单编号、时间、规模等不同信息
        separate_by_slash = separate_by_space[0].split("_")

        # 获取订单类型信息（如"Buy"表示买入，"Sell"表示卖出）
        order_type = separate_by_slash[0]
        # 获取订单编号相关信息（具体含义需结合业务场景确定）
        order_id = separate_by_slash[1]
        # 获取订单时间信息，用于后续按时间维度对数据进行整理和填充
        order_time = separate_by_slash[2]
        # 获取订单规模信息（如"ExtraLarge"、"Large"等表示不同规模的订单）
        order_size = separate_by_slash[3]
        # 将订单数量字段从字符串转换为浮点数类型，用于后续赋值等操作
        order_qty = float(separate_by_colon[0])
        # 将订单价格字段从字符串转换为浮点数类型，并去除可能存在的空白字符（比如字符串末尾的空格等），同样用于后续计算和赋值
        order_price = float(separate_by_colon[1].strip())

        # 将当前订单的时间信息添加到时间列表中，用于记录所有出现过的时间情况
        time_list.append(order_time)

        # 判断当前订单的时间是否已经存在于price_table的'时间范围'列中，如果不存在，则添加新的一行数据，该行只包含当前的时间信息
        if order_time not in price_table["时间范围"].values:
            new_row = pd.DataFrame({"时间范围": [order_time]})
            price_table = pd.concat([price_table, new_row], ignore_index=True)

        # 如果订单类型是买入（"Buy"）
        if order_type == "Buy":
            # 如果订单规模是超大单（"ExtraLarge"）
            if order_size == "ExtraLarge":
                # 在price_table中找到对应时间范围的行，将超大买单成交量列的值设置为当前订单的数量
                price_table.loc[
                    price_table["时间范围"] == order_time, "超大买单成交量"
                ] = order_qty
                # 同样，将超大买单成交额列的值设置为当前订单的价格（这里可能业务逻辑上不太准确，成交额一般应该是数量乘以价格，不过需结合实际情况看，也许后续还有其他处理）
                price_table.loc[
                    price_table["时间范围"] == order_time, "超大买单成交额"
                ] = order_price
            # 如果订单规模是大单（"Large"）
            elif order_size == "Large":
                price_table.loc[
                    price_table["时间范围"] == order_time, "大买单成交量"
                ] = order_qty
                price_table.loc[
                    price_table["时间范围"] == order_time, "大买单成交额"
                ] = order_price
            # 如果订单规模是中单（"Medium"）
            elif order_size == "Medium":
                price_table.loc[
                    price_table["时间范围"] == order_time, "中买单成交量"
                ] = order_qty
                price_table.loc[
                    price_table["时间范围"] == order_time, "中买单成交额"
                ] = order_price
            # 如果订单规模是小单（"Small"）
            elif order_size == "Small":
                price_table.loc[
                    price_table["时间范围"] == order_time, "小买单成交量"
                ] = order_qty
                price_table.loc[
                    price_table["时间范围"] == order_time, "小买单成交额"
                ] = order_price
        # 如果订单类型是卖出（"Sell"）
        elif order_type == "Sell":
            # 如果订单规模是超大单（"ExtraLarge"）
            if order_size == "ExtraLarge":
                price_table.loc[
                    price_table["时间范围"] == order_time, "超大卖单成交量"
                ] = order_qty
                price_table.loc[
                    price_table["时间范围"] == order_time, "超大卖单成交额"
                ] = order_price
            # 如果订单规模是大单（"Large"）
            elif order_size == "Large":
                price_table.loc[
                    price_table["时间范围"] == order_time, "大卖单成交量"
                ] = order_qty
                price_table.loc[
                    price_table["时间范围"] == order_time, "大卖单成交额"
                ] = order_price
            # 如果订单规模是中单（"Medium"）
            elif order_size == "Medium":
                price_table.loc[
                    price_table["时间范围"] == order_time, "中卖单成交量"
                ] = order_qty
                price_table.loc[
                    price_table["时间范围"] == order_time, "中卖单成交额"
                ] = order_price
            # 如果订单规模是小单（"Small"）
            elif order_size == "Small":
                price_table.loc[
                    price_table["时间范围"] == order_time, "小卖单成交量"
                ] = order_qty
                price_table.loc[
                    price_table["时间范围"] == order_time, "小卖单成交额"
                ] = order_price

In [57]:
columns_list_2 = columns_list.copy()
columns_list_2.remove('时间范围')
price_table.fillna(0, inplace=True)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9892\3023856724.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  price_table.fillna(0, inplace=True)


In [58]:
type(price_table['超大卖单成交量'][0])

numpy.float64

In [60]:
# 遍历时间列表中的每个时间元素，这里的时间列表（time_list）应该是之前代码中收集的所有交易数据涉及到的不同时间范围
for time in time_list:
    # 计算每个时间范围内的主力流入数值，主力流入定义为超大买单成交额与大买单成交额之和
    # 通过.loc方法定位到price_table中'时间范围'列等于当前时间的行，然后将'主力流入'列的值设置为对应行的'超大买单成交额'与'大买单成交额'列的值之和
    price_table.loc[price_table["时间范围"] == time, "主力流入"] = (
        price_table.loc[price_table["时间范围"] == time, "超大买单成交额"]
        + price_table.loc[price_table["时间范围"] == time, "大买单成交额"]
    )
    # 计算每个时间范围内的主力流出数值，主力流出定义为超大卖单成交额与大卖单成交额之和
    # 同样使用.loc方法定位相应行，将'主力流出'列的值设置为对应行的'超大卖单成交额'与'大卖单成交额'列的值之和
    price_table.loc[price_table["时间范围"] == time, "主力流出"] = (
        price_table.loc[price_table["时间范围"] == time, "超大卖单成交额"]
        + price_table.loc[price_table["时间范围"] == time, "大卖单成交额"]
    )
    # 计算每个时间范围内的主力净流入数值，主力净流入等于主力流入减去主力流出
    # 还是通过.loc方法先定位到相应行，将'主力净流入'列的值设置为同一行中'主力流入'列的值减去'主力流出'列的值
    price_table.loc[price_table["时间范围"] == time, "主力净流入"] = (
        price_table.loc[price_table["时间范围"] == time, "主力流入"]
        - price_table.loc[price_table["时间范围"] == time, "主力流出"]
    )


# 按照'时间范围'列的值对price_table进行升序排序，使得数据按照时间先后顺序排列，方便查看和后续分析等操作
price_table.sort_values(by="时间范围", ascending=True, inplace=True)
# 重置price_table的索引，丢弃原来的索引（drop=True表示原来的索引不再保留），重新生成从0开始的连续索引，保证索引的顺序与排序后的数据顺序一致
price_table.reset_index(drop=True, inplace=True)
# 将整理好的price_table数据保存为CSV文件，文件名为'price_table_sorted.csv'，设置index=False表示不将索引列写入文件，encoding='gbk'指定文件编码为GBK，以适应可能的中文等字符存储需求
price_table.to_csv("price_table_sorted.csv", index=False, encoding="gbk")